In [1]:
import pandas as pd
import numpy as np

In [2]:
loans=pd.read_csv('lending-club-data.csv')
loans['safe_loans']= loans['bad_loans'].apply(lambda x: 1 if x==0 else -1)

C:\Users\Keulando\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
foo=features+[target]

In [4]:
loans=loans.loc[:,foo]
loans.head()

,grade,term,home_ownership,emp_length,safe_loans
0,B,36 months,RENT,10+ years,1
1,C,60 months,RENT,< 1 year,-1
2,C,36 months,RENT,10+ years,1
3,C,36 months,RENT,10+ years,1
4,A,36 months,RENT,3 years,1


In [5]:
loans=pd.get_dummies(loans)
loans.head()

,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,...,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,emp_length_n/a
0,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [6]:
train_idx=pd.read_json('module-6-assignment-train-idx.json',      typ='series')
valid_idx=pd.read_json('module-6-assignment-validation-idx.json', typ='series')

train=loans.iloc[train_idx]
valid=loans.iloc[valid_idx]

In [7]:
train.head()

,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,...,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,emp_length_n/a
1,-1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
6,-1,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
7,-1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
10,-1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
12,-1,0,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


# Early stopping methods for decision trees

1) Reached a maximum depth. (set by parameter max_depth).

2) Reached a minimum node size. (set by parameter min_node_size).

3) Don't split if the gain in error reduction is too small. (set by parameter min_error_reduction).

## wirte function for 2nd condiiton 

In [8]:
def reached_min_node_size(data, min_node_size):
    if len(data) <= min_node_size:
        return True

## write funct for 3rd condition

In [9]:
def error_reduction(error_before_split, error_after_split):
    error=error_before_split-error_after_split
    return error

## use other funct from previous ass

In [10]:
def intermediate_node_num_mistakes(labels_in_node):
    #if labels in node is empty return 0
    if len(labels_in_node)==0:
        return 0
    
    pos_count  = (labels_in_node ==1).sum()
    neg_count = (labels_in_node ==-1).sum()
    
    #return mistakes
    return neg_count if pos_count > neg_count else pos_count

In [11]:
def best_splittin_feature(data, features, target):
    
    target_values = data[target]
    best_feature  = None #keep track of best feature
    best_error    = 10 #keep track best error
    
    num_data_points=float(len(data))
    
    for feature in features:
        left_split  = data[data[feature]==0]
        right_split = data[data[feature]==1]
        
        left_mistakes  = intermediate_node_num_mistakes(right_split[target])
        right_mistakes = intermediate_node_num_mistakes(left_split[target])
        
        error = (left_mistakes + right_mistakes)/ num_data_points
        
        if error < best_error:
            best_feature = feature
            best_error   = error
   
    return best_feature

In [12]:
def create_leaf(target_values):
    leaf = {'splitting_feature': None,
           'left': None,
           'right': None,
           'is_leaf': True}
    
    #count num data points
    num_pos=len(target_values[target_values ==1])
    num_neg= len(target_values[target_values == -1])
    
    if num_pos > num_neg:
        leaf['prediction']= 1
    else:    
        leaf['prediction']= -1
        
    return leaf

# Incorporating new early  conditions in binary decision tree implementation

In [13]:
def decision_tree_create(data, features, target, current_depth=0, max_depth=10,
                        min_node_size=1, min_error_reduction=0.0):
    
    remaining_features=features[:]
    target_values=data[target]
    
    print('---------------------------------------------------------')
    print('Subtree, dept : {} ({} data points)'.format(current_depth, len(target_values)))
    
    #stopping condition 1 'data point are from the same class'
    if intermediate_node_num_mistakes(target_values)==0:
        print('Stopping condtion 1 reached. All data points have th same target value.')
         #If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    #stopping condition 2 'no more features to split on'
    if len(remaining_features)  == 0:
        print(' stopping condition 2 reached. No remaining features.')
        return create_leaf(target_values)
    
    #early stopping condition 1; reached max limit
    if current_depth >= max_depth:
        print(' Early stopping condition reached. Reached maximu depth')
        return create_leaf(target_values)
    
    #early stopping cond 2: reached min node size
    if  reached_min_node_size(data, min_node_size) == True:
        print('Early stopping condition 2 reached. Reached min node size')
        return create_leaf(target_values)
    
    
    # find best feature
    splitting_feature=best_splittin_feature(data, features, target)
    
    #split on the best feature
    left_split  = data[data[splitting_feature]==0]
    right_split = data[data[splitting_feature]==1]
    
    # early stopping cond 3. Min error reduction
    error_before_split = intermediate_node_num_mistakes(target_values)/float(len(data))
    left_mistakes      = intermediate_node_num_mistakes(left_split[target])
    right_mistakes     = intermediate_node_num_mistakes(right_split[target])
    error_after_split  = (left_mistakes + right_mistakes)/float(len(data))
    
    if error_reduction(error_before_split, error_after_split) <= min_error_reduction:
        print(' Early stopping condition 3 reached. Minimum erorr reduction')
        return create_leaf(target_values)
     
    remaining_features.drop(splitting_feature, axis=1, inplace=True)
    print('split on feature {}. {}, {}'.format(splitting_feature, len(left_split), len(right_split)))
    
    #create leaf node if split is perfect
    if len(left_split) == len(data):
        print('creating leaf node')
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print('Creating lead node')
        return create_leaf(right_split[target])
    
    #repeat on left and right
    
    left_tree  = decision_tree_create(left_split,  remaining_features,
                                      target, current_depth +1, max_depth,
                                      min_node_size, min_error_reduction)
    right_tree = decision_tree_create(right_split, remaining_features,
                                      target, current_depth +1, max_depth,
                                      min_node_size, min_error_reduction)
 
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

# train model

In [67]:
features_bin=list(loans.columns[1:])
target= 'safe_loans'

['grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'term_ 36 months',
 'term_ 60 months',
 'home_ownership_MORTGAGE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'emp_length_1 year',
 'emp_length_10+ years',
 'emp_length_2 years',
 'emp_length_3 years',
 'emp_length_4 years',
 'emp_length_5 years',
 'emp_length_6 years',
 'emp_length_7 years',
 'emp_length_8 years',
 'emp_length_9 years',
 'emp_length_< 1 year',
 'emp_length_n/a']

In [14]:
features_bin=loans.iloc[:,1:]
target= 'safe_loans'


In [17]:

my_tree_new=decision_tree_create(train, features_bin, target, current_depth=0,
                             max_depth=6, min_node_size=100, min_error_reduction=0.0)

---------------------------------------------------------
Subtree, dept : 0 (37224 data points)
split on feature term_ 36 months. 9223, 28001
---------------------------------------------------------
Subtree, dept : 1 (9223 data points)


C:\Users\Keulando\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


split on feature grade_A. 9122, 101
---------------------------------------------------------
Subtree, dept : 2 (9122 data points)
 Early stopping condition 3 reached. Minimum erorr reduction
---------------------------------------------------------
Subtree, dept : 2 (101 data points)
split on feature emp_length_n/a. 96, 5
---------------------------------------------------------
Subtree, dept : 3 (96 data points)
Early stopping condition 2 reached. Reached min node size
---------------------------------------------------------
Subtree, dept : 3 (5 data points)
Early stopping condition 2 reached. Reached min node size
---------------------------------------------------------
Subtree, dept : 1 (28001 data points)
split on feature grade_D. 23300, 4701
---------------------------------------------------------
Subtree, dept : 2 (23300 data points)
split on feature grade_E. 22024, 1276
---------------------------------------------------------
Subtree, dept : 3 (22024 data points)
split on f

In [18]:
my_tree_old=decision_tree_create(train, features_bin, target, max_depth=6,
                                min_node_size=0, min_error_reduction=-1)

---------------------------------------------------------
Subtree, dept : 0 (37224 data points)
split on feature term_ 36 months. 9223, 28001
---------------------------------------------------------
Subtree, dept : 1 (9223 data points)


C:\Users\Keulando\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


split on feature grade_A. 9122, 101
---------------------------------------------------------
Subtree, dept : 2 (9122 data points)
split on feature grade_B. 8074, 1048
---------------------------------------------------------
Subtree, dept : 3 (8074 data points)
split on feature grade_C. 5884, 2190
---------------------------------------------------------
Subtree, dept : 4 (5884 data points)
split on feature grade_D. 3826, 2058
---------------------------------------------------------
Subtree, dept : 5 (3826 data points)
split on feature grade_E. 1693, 2133
---------------------------------------------------------
Subtree, dept : 6 (1693 data points)
 Early stopping condition reached. Reached maximu depth
---------------------------------------------------------
Subtree, dept : 6 (2133 data points)
 Early stopping condition reached. Reached maximu depth
---------------------------------------------------------
Subtree, dept : 5 (2058 data points)
split on feature grade_E. 2058, 0
creat

# Predictions

In [43]:
def classify(tree, x, annotate= False):
    
    if tree['is_leaf']:
        if annotate:
            print('At leaf, predicting {}'.format(tree['prediction']))
        return tree['prediction']
    else:
        #split on feature
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
            print('Split on {} = {}'.format(tree['splitting_feature'],split_feature_value))
        if split_feature_value == 0:
                return classify(tree['left'],x, annotate)
                    
        else:
            return classify(tree['right'], x, annotate)

In [23]:
#print(valid.iloc[0])
print('True Class is:      {}'.format(valid.iloc[0][0]))
print('Predicted class is: {} '.format(classify(my_tree_new, valid.iloc[0])))

True Class is:      -1
Predicted class is: -1 


# Q3 is the prediction path shorter, longer, or the same as the prediction path using my_decision_tree_old that ignored the early stopping conditions 2 and 3?

on valid[0[

In [24]:
classify(my_tree_new, valid.iloc[0], annotate=True)

Split on term_ 36 months = 0
Split on grade_A = 0
At leaf, predicting -1


-1

In [25]:
# no early stopping condition
classify(my_tree_old, valid.iloc[0], annotate=True)

Split on term_ 36 months = 0
Split on grade_A = 0
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 1
At leaf, predicting -1


-1

In [26]:
classify(my_tree_new, valid.iloc[1123], annotate=True)

Split on term_ 36 months = 1
Split on grade_D = 0
Split on grade_E = 0
Split on grade_F = 0
Split on emp_length_n/a = 0
Split on grade_G = 1
At leaf, predicting -1


-1

# Evaluate

In [27]:
def evaluate_clf_error(tree, data):
    prediction = data.apply(lambda x: classify(tree, x),axis=1)
    mistakes   = ( prediction != data[target]).sum() / float(len(data))
    return mistakes

In [28]:
evaluate_clf_error(my_tree_new, valid)

0.38367083153813014

In [29]:
evaluate_clf_error(my_tree_old, valid)

0.3837785437311504

# exploring effect of max_depth

In [31]:
model_1=decision_tree_create(train, features_bin, target, current_depth=0,
                             max_depth=2, min_node_size=0, min_error_reduction=-1 )

model_2=decision_tree_create(train, features_bin, target, current_depth=0,
                             max_depth=6, min_node_size=0, min_error_reduction=-1 )

model_3=decision_tree_create(train, features_bin, target, current_depth=0,
                             max_depth=14, min_node_size=0, min_error_reduction=-1 )

---------------------------------------------------------
Subtree, dept : 0 (37224 data points)
split on feature term_ 36 months. 9223, 28001
---------------------------------------------------------
Subtree, dept : 1 (9223 data points)


C:\Users\Keulando\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


split on feature grade_A. 9122, 101
---------------------------------------------------------
Subtree, dept : 2 (9122 data points)
 Early stopping condition reached. Reached maximu depth
---------------------------------------------------------
Subtree, dept : 2 (101 data points)
 Early stopping condition reached. Reached maximu depth
---------------------------------------------------------
Subtree, dept : 1 (28001 data points)
split on feature grade_D. 23300, 4701
---------------------------------------------------------
Subtree, dept : 2 (23300 data points)
 Early stopping condition reached. Reached maximu depth
---------------------------------------------------------
Subtree, dept : 2 (4701 data points)
 Early stopping condition reached. Reached maximu depth
---------------------------------------------------------
Subtree, dept : 0 (37224 data points)
split on feature term_ 36 months. 9223, 28001
---------------------------------------------------------
Subtree, dept : 1 (9223 da

split on feature grade_C. 34, 0
creating leaf node
---------------------------------------------------------
Subtree, dept : 5 (45 data points)
split on feature grade_C. 45, 0
creating leaf node
---------------------------------------------------------
Subtree, dept : 2 (101 data points)
split on feature emp_length_n/a. 96, 5
---------------------------------------------------------
Subtree, dept : 3 (96 data points)
split on feature emp_length_< 1 year. 85, 11
---------------------------------------------------------
Subtree, dept : 4 (85 data points)
split on feature grade_B. 85, 0
creating leaf node
---------------------------------------------------------
Subtree, dept : 4 (11 data points)
split on feature grade_B. 11, 0
creating leaf node
---------------------------------------------------------
Subtree, dept : 3 (5 data points)
split on feature grade_B. 5, 0
creating leaf node
---------------------------------------------------------
Subtree, dept : 1 (28001 data points)
split on

split on feature home_ownership_OWN. 9, 2
---------------------------------------------------------
Subtree, dept : 6 (9 data points)
split on feature grade_A. 9, 0
creating leaf node
---------------------------------------------------------
Subtree, dept : 6 (2 data points)
Stopping condtion 1 reached. All data points have th same target value.
---------------------------------------------------------
Subtree, dept : 3 (1276 data points)
split on feature grade_A. 1276, 0
creating leaf node
---------------------------------------------------------
Subtree, dept : 2 (4701 data points)
split on feature grade_A. 4701, 0
creating leaf node


In [196]:

error_dict= {'Model 1':{'max_depth':int(2),
                        'min_node_size':0, 
                        'min_error_reduction':-1,
                        'Train error':evaluate_clf_error(model_1, train),
                        'Validation error': evaluate_clf_error(model_1, valid)},
          'Model 2':{'max_depth':6,
                    'min_node_size':0, 
                    'min_error_reduction':-1,
                    'Train error':evaluate_clf_error(model_2, train),
                    'Validation error': evaluate_clf_error(model_2, valid)},
          'Model 3':{'max_depth':14,
                    'min_node_size':0, 
                    'min_error_reduction':-1,
                    'Train error':evaluate_clf_error(model_3, train),
                    'Validation error': evaluate_clf_error(model_3, valid)}}

error_table=pd.DataFrame(error_dict)
error_table

,Model 1,Model 2,Model 3
Train error,0.400038,0.381850,0.376182
Validation error,0.398104,0.383779,0.377316
max_depth,2.000000,6.000000,14.000000
min_error_reduction,-1.000000,-1.000000,-1.000000
min_node_size,0.000000,0.000000,0.000000


# Measuring the complexity of the tree


In [56]:
def counts_leaf(tree):
    if tree['is_leaf']:
        return 1
    return counts_leaf(tree['left']) + counts_leaf(tree['right'])

In [59]:
complexity_dict= {'Model 1':{'N. leafs': counts_leaf(model_1)},
                 'Model 2':{'N. leafs': counts_leaf(model_2)},
                 'Model 3':{'N. leafs': counts_leaf(model_3)}}

complexity_table= pd.DataFrame(complexity_dict)
complexity_table

,Model 1,Model 2,Model 3
N. leafs,4,19,41


# Exploring the effect of min_error


In [60]:
model_4=decision_tree_create(train, features_bin, target, current_depth=0,
                             max_depth=6, min_node_size=0, min_error_reduction=-1 )

model_5=decision_tree_create(train, features_bin, target, current_depth=0,
                             max_depth=6, min_node_size=0, min_error_reduction=0 )

model_6=decision_tree_create(train, features_bin, target, current_depth=0,
                             max_depth=6, min_node_size=0, min_error_reduction=5 )

---------------------------------------------------------
Subtree, dept : 0 (37224 data points)
split on feature term_ 36 months. 9223, 28001
---------------------------------------------------------
Subtree, dept : 1 (9223 data points)


C:\Users\Keulando\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


split on feature grade_A. 9122, 101
---------------------------------------------------------
Subtree, dept : 2 (9122 data points)
split on feature grade_B. 8074, 1048
---------------------------------------------------------
Subtree, dept : 3 (8074 data points)
split on feature grade_C. 5884, 2190
---------------------------------------------------------
Subtree, dept : 4 (5884 data points)
split on feature grade_D. 3826, 2058
---------------------------------------------------------
Subtree, dept : 5 (3826 data points)
split on feature grade_E. 1693, 2133
---------------------------------------------------------
Subtree, dept : 6 (1693 data points)
 Early stopping condition reached. Reached maximu depth
---------------------------------------------------------
Subtree, dept : 6 (2133 data points)
 Early stopping condition reached. Reached maximu depth
---------------------------------------------------------
Subtree, dept : 5 (2058 data points)
split on feature grade_E. 2058, 0
creat

 Early stopping condition 3 reached. Minimum erorr reduction


In [197]:

foo_dict= {'Model 4':{'max_depth':6,
                      'min_node_size':0, 
                      'min_error_reduction':-1,
                      'Train error': evaluate_clf_error(model_4, train),
                      'Validation error': evaluate_clf_error(model_4, valid),
                     'N. leafs': counts_leaf(model_4)},
           'Model 5':{'max_depth':6,
                      'min_node_size':0, 
                      'min_error_reduction':0,
                      'Train error': evaluate_clf_error(model_5, train),
                      'Validation error': evaluate_clf_error(model_5, valid),
                     'N. leafs': counts_leaf(model_5)},
          'Model 6':{'max_depth':6,
                      'min_node_size':0, 
                      'min_error_reduction':5,
                      'Train error': evaluate_clf_error(model_6, train),
                     'Validation error': evaluate_clf_error(model_6, valid),
                     'N. leafs': counts_leaf(model_6)}}

foo_table=pd.DataFrame(foo_dict)
foo_table

,Model 4,Model 5,Model 6
N. leafs,19.000000,13.000000,1.000000
Train error,0.381850,0.381958,0.496346
Validation error,0.383779,0.383779,0.503447
max_depth,6.000000,6.000000,6.000000
min_error_reduction,-1.000000,0.000000,5.000000
min_node_size,0.000000,0.000000,0.000000


# Exploring the effect of min_node_size


In [64]:
model_7=decision_tree_create(train, features_bin, target, current_depth=0,
                             max_depth=6, min_node_size=0, min_error_reduction=-1 )

model_8=decision_tree_create(train, features_bin, target, current_depth=0,
                             max_depth=6, min_node_size=2000, min_error_reduction=-1 )

model_9=decision_tree_create(train, features_bin, target, current_depth=0,
                             max_depth=6, min_node_size=50000, min_error_reduction=-1 )

---------------------------------------------------------
Subtree, dept : 0 (37224 data points)
split on feature term_ 36 months. 9223, 28001
---------------------------------------------------------
Subtree, dept : 1 (9223 data points)


C:\Users\Keulando\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


split on feature grade_A. 9122, 101
---------------------------------------------------------
Subtree, dept : 2 (9122 data points)
split on feature grade_B. 8074, 1048
---------------------------------------------------------
Subtree, dept : 3 (8074 data points)
split on feature grade_C. 5884, 2190
---------------------------------------------------------
Subtree, dept : 4 (5884 data points)
split on feature grade_D. 3826, 2058
---------------------------------------------------------
Subtree, dept : 5 (3826 data points)
split on feature grade_E. 1693, 2133
---------------------------------------------------------
Subtree, dept : 6 (1693 data points)
 Early stopping condition reached. Reached maximu depth
---------------------------------------------------------
Subtree, dept : 6 (2133 data points)
 Early stopping condition reached. Reached maximu depth
---------------------------------------------------------
Subtree, dept : 5 (2058 data points)
split on feature grade_E. 2058, 0
creat

In [296]:
node_dict= {'Model 7':{'max_depth':6,
                      'min_node_size':0, 
                      'min_error_reduction':-1,
                       'Train error': evaluate_clf_error(model_7, train),
                       'Validation error': evaluate_clf_error(model_7, valid),
                     'N. leafs': counts_leaf(model_7)},
           'Model 8':{'max_depth':6,
                      'min_node_size':2000, 
                      'min_error_reduction':-1,
                      'Train error': evaluate_clf_error(model_8, train),
                      'Validation error': evaluate_clf_error(model_8, valid),
                     'N. leafs': counts_leaf(model_8)},
          'Model 9':{'max_depth':np.around(6,1),
                      'min_node_size':50000, 
                      'min_error_reduction':-1,
                     'Train error': evaluate_clf_error(model_9, train),
                     'Validation error': evaluate_clf_error(model_9, valid),
                     'N. leafs': counts_leaf(model_9)}}

table_node= pd.DataFrame(node_dict)

In [297]:
table_node

,Model 7,Model 8,Model 9
N. leafs,19.000000,12.000000,1.000000
Train error,0.381850,0.383704,0.496346
Validation error,0.383779,0.384533,0.503447
max_depth,6.000000,6.000000,6.000000
min_error_reduction,-1.000000,-1.000000,-1.000000
min_node_size,0.000000,2000.000000,50000.000000


In [299]:
from itertools import chain
dee= dict(chain(error_dict.items(), foo_dict.items(), node_dict.items()))
dd=pd.DataFrame(dee)
dd.iloc[0,:3] = complexity_table.iloc[0,:]
dd=dd.T.round(3)

In [301]:
dd

,N. leafs,Train error,Validation error,max_depth,min_error_reduction,min_node_size
Model 1,4.0,0.400,0.398,2.0,-1.0,0.0
Model 2,19.0,0.382,0.384,6.0,-1.0,0.0
Model 3,41.0,0.376,0.377,14.0,-1.0,0.0
Model 4,19.0,0.382,0.384,6.0,-1.0,0.0
Model 5,13.0,0.382,0.384,6.0,0.0,0.0
Model 6,1.0,0.496,0.503,6.0,5.0,0.0
Model 7,19.0,0.382,0.384,6.0,-1.0,0.0
Model 8,12.0,0.384,0.385,6.0,-1.0,2000.0
Model 9,1.0,0.496,0.503,6.0,-1.0,50000.0
